In [ ]:
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_create_report
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = ''
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = ""

outdir = setup("20220628_qc_from_june_briefing.ipynb")

In [ ]:
import pandas as pd
obs_fpath = "covid_model/analysis/20220628_july_gov_briefing/CDPHE Variant Data (manually transcribed and limited to model variants).csv"
obs = pd.read_csv(obs_fpath, parse_dates=['Date']).fillna(0)
obs['Date'] = [d.date() for d in obs['Date']]
obs = obs.set_index('Date')

In [ ]:
seeds = [
    {"param": "omicron_seed","attrs": None,"vals": {"2020-01-01": 0, "2021-12-01": 1000, "2021-12-05": 3000, "2021-12-20": 1000, "2021-12-31": 0},"desc": ""},
    {"param": "ba2_seed",    "attrs": None,"vals": {"2020-01-01": 0, "2021-12-23": 2, "2022-03-05": 300, "2022-03-9": 400, "2022-03-20": 0},"desc": ""},
    {"param": "ba2121_seed", "attrs": None,"vals": {"2020-01-01": 0, "2022-03-10": 2, "2022-03-25": 30, "2022-05-20": 0},"desc": ""},
    {"param": "ba45_seed", "attrs": None,"vals": {"2020-01-01": 0, "2022-04-15": 10, "2022-05-15": 20, "2022-06-01": 0},"desc": ""},
    {"param": "immune_escape", "from_attrs": {"immun": "strong", "variant": ["none", "wildtype", "alpha", "delta"]}, "to_attrs": {"variant": ["omicron", "ba2", "ba2121", "ba45"]},  "vals": {"2020-01-01": 0.1},  "desc":  ""},
    {"param": "immune_escape", "from_attrs": {"immun": "weak", "variant": ["none", "wildtype", "alpha", "delta"]}, "to_attrs": {"variant": ["omicron", "ba2", "ba2121", "ba45"]},  "vals": {"2020-01-01": 0.75},  "desc": ""},
    {"param": "immune_escape", "from_attrs": {"immun": "strong", "variant": ["omicron", "ba2"]}, "to_attrs": {"variant": ["ba2121"]},  "vals": {"2020-01-01": 0.03},  "desc": ""},
    {"param": "immune_escape", "from_attrs": {"immun": "weak", "variant": ["omicron", "ba2"]}, "to_attrs": {"variant": ["ba2121"]},  "vals": {"2020-01-01": 0.25},  "desc": ""},
    {"param": "immune_escape", "from_attrs": {"immun": "strong", "variant": ["omicron", "ba2", "ba2121"]}, "to_attrs": {"variant": ["ba45"]},  "vals": {"2020-01-01": 0.06},  "desc": "strong"},
    {"param": "immune_escape", "from_attrs": {"immun": "weak", "variant": ["omicron", "ba2", "ba2121"]}, "to_attrs": {"variant": ["ba45"]},  "vals": {"2020-01-01": 0.50},  "desc": "weak"},
    {"param": "severe_immunity", "attrs": {"immun": ["strong", "weak"]}, "vals": {"2020-01-01": 0.975}, "desc": "Vaccinated immunity vs severe disease is very high (among vacc. successes)."},
    {"param": "severe_immunity", "attrs": {"immun": ["strong", "weak"], "vacc": "none"}, "vals": {"2020-01-01": 0.5}, "desc": "Prior infection immunity vs severe disease is much weaker than vaccination immunity."},
    {"param": "severe_immunity",  "attrs": {"variant": "delta"}, "mults": {"2020-01-01": 0.9}, "desc": "Deltas reduces immunity to severe disease."},
    {"param": "severe_immunity",  "attrs": {"vacc": ["shot1", "shot2"], "variant": ["omicron", "ba2", "ba2121", "ba45"]}, "mults": {"2020-01-01": 0.7}, "desc": "Omicron(s) reduce immunity to severe disease."},
    {"param": "severe_immunity",  "attrs": {"vacc": "shot3", "variant": ["omicron", "ba2", "ba2121", "ba45"]}, "mults": {"2020-01-01": 0.9}, "desc": "Omicron(s) reduce immunity to severe disease."},
]

model_args = {
    'base_spec_id': 3014,
    'params_defs': json.load(open('covid_model/analysis/20220628_july_gov_briefing/qc_params.json')) + seeds,
    #'region_defs': 'covid_model/input/region_definitions.json',
    #'vacc_proj_params': 'covid_model/input/vacc_proj_params.json',
    #'regions': ['co'],
    #'mobility_mode': None,
    #'start_date': '2020-01-24',
    #'end_date': '2022-09-15',
    #'max_step_size': 1.0,
    #'ode_method': 'RK45',
    'hosp_reporting_frac': {"2020-01-01": 1, "2022-03-01": 0.8, "2022-03-15": 0.66, "2022-03-30": 0.5}
}
# create initial solution so we don't have to fit the whole window
model = CovidModel(**model_args)
model.prep(outdir=outdir)
model.solve_seir()
# now fit, but only the period of time which is different from the base spec id
fit_args = {
    'fit_start_date': '2021-11-01',
    #'fit_end_date': '2022-05-01',
    'tc_min': 0.0,
    'tc_max': 0.999,
    'tc_window_size': 14,
    'tc_window_batch_size': 6,
    'tc_batch_increment': 2,
    'last_tc_window_min_size': 14,
    'outdir': outdir
}
model = do_single_fit(**fit_args, base_model = model, prep_model=False)

In [ ]:
model.solve_seir()
soln = model.solution_sum_df(['seir', 'variant'])

In [ ]:
start_date = dt.datetime.strptime('2022-01-01', '%Y-%m-%d').date()
end_date = dt.datetime.strptime('2022-09-15', '%Y-%m-%d').date()
rows = (soln.index.get_level_values(0) >= start_date) & (soln.index.get_level_values(0) <= end_date)
cols = [any(colname == seir for seir in ['Ih', 'I', 'A']) for colname in soln.columns.get_level_values(0)]
infected_by_variant = soln.loc[rows, cols].groupby('variant', axis=1).sum()

In [ ]:
infected_by_variant.plot(figsize=(10, 5))

In [ ]:
infected_by_variant.div(infected_by_variant.sum(axis=1), axis=0).plot(figsize=(10, 5))

In [ ]:
obs_rows = (obs.index.values >= start_date) & (obs.index.values <= end_date)
obs['BA45'] = obs['BA.4'] + obs['BA.5']
obs.drop(columns=['BA.4', 'BA.5']).loc[obs_rows,].plot(figsize=(10,5))

In [ ]:
infected_by_variant.sum(axis=1).plot(figsize=(10, 5))

In [ ]:
model.modeled_vs_observed_hosps().plot(figsize=(10,5))

In [ ]:
do_create_report(model, outdir)